<a href="https://colab.research.google.com/github/MinKimIP/IPA-public/blob/master/uspto-tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Notebook

Date

Description

In [0]:
%%capture
!pip install pandas --upgrade
!pip install pandas-profiling[notebook,html] --upgrade

In [0]:
import pandas as pd